In [1]:
#Number of qubit pairs. For N<=7 computations will take a reasonable time.
N=5

In [2]:
#Get the l least significant bits of a nonnegative integer n.
#This is used to create binary vectors
def bitfield(n,l):
    return [(n >> (l-1-i))%2 for i in range(l)]


#standard basis vectors of length N-1
e=[bitfield(1 << i,N-1) for i in range(N-1)]


#construct the power set of a set s
def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]
        

In [3]:
#Construct a 2N x N+1 submatrix of a symplectic matrix (corresponding to the  pillars)
#The last N-1 columns correspond to the basis
def Pillars(A,a,b):
    M=[[1,0]+a]+[  [b[i],a[i]]+[a[i]*b[j]^^A[i][j] for j in range(N-1)] for i in range(N-1)]
    M+=[[0,1]+b]+[[0,0]+e[N-2-i] for i in range(N-1)]
    return M

In [3]:
#Construct the polynomials p(basis) and q(pillars) for a given (2NxN-1 submatrix of) a symplectic matrix

def poly(M):
    p=[0 for x in range(N+1)]
    q=[0 for x in range(N+1)]
    
    for x in powerset(range(N-1)):
        k=0
        for i in range(N):
            if (sum([M[i][j+2] for j in x])%2==0) and (sum([M[i+N][j+2] for j in x])%2==0): k+=1
        p[k]+=1  
        
    for x in powerset(range(N+1)):
        k=0
        for i in range(N):
            if (sum([M[i][j] for j in x])%2==0) and (sum([M[i+N][j] for j in x])%2==0): k+=1
        q[k]+=1              
                  
    return (tuple(p),tuple(q))

#Create symbolic expression in terms of F (the fidelity) from poly, i.e. output from poly function

def symb_poly(poly):
    p, q = poly
    n = len(p) - 1
    F = var('F')
    
    prob = 0
    num = 0
    
    for k in range(n+1):
        prob += q[k]*(F**k)*((1-F)/3)**(n-k)
        num += p[k]*(F**k)*((1-F)/3)**(n-k)
    fidelity = num/prob
    
    return prob.full_simplify(), fidelity.full_simplify()

In [ ]:

#generate the unlabeled graphs on N-1 nodes and get their adjacency matrices.
g=graphs(N-1)
print(N)
import random
AG=[G.adjacency_matrix() for G in g]
random.shuffle(AG)
# AG = [G for G in AG if are_sub_blocks_zero(G, N-1)]
# AG = [G if for G in g]
# AG = [0*identity_matrix(N-1)]

11


In [ ]:
#pairs (a,b) of N-1 bit vectors with a <= b <= a XOR b. This reduces some symmetry. 
ab_pairs=[[bitfield(a,N-1),bitfield(b,N-1)] for a in range(2^(N-1)) for b in range(2^(N-1)) if (a<=b and b<=a^^b)]
random.shuffle(ab_pairs)

In [ ]:
#Collect all polynomials for all relevant symplectic matrices
#Each combination in AG x ab_pairs results in one symplectic matrix to check
polies=dict()
graphnr=1
tot=len(AG)
F_max = 0
print("Computing statistics for %d x %d = %d symplectic matrices" % (tot,len(ab_pairs), tot*len(ab_pairs)))
for A in AG:
    print("Completed %d of %d" % (graphnr, tot))
    graphnr+=1
    for a, b in ab_pairs:
        polynomials = poly(Pillars(A, a, b))
        if polynomials not in polies:
            F = symb_poly(polynomials)[1](F=0.7)
            if F > F_max:
                F_max = F
                print(F)
                print(symb_poly(polynomials)[1])
                print(symb_poly(polynomials)[1].series(F=1,8))
            polies[polynomials] = (A, a, b)
            print('Number of distinct statistics so far: ', len(polies))
            save(polies, 'p+F+a+b+B+n5.sobj')
    #polies2+=[(poly(Pillars(A,a,b)), (A, a, b)) for [a,b] in ab_pairs]
    
distinct_polies=set(polies)
save(polies, 'p+F+a+b+B+n5.sobj')
print("Number of distinct statistics:", len(distinct_polies))

In [2]:
# Find maximum fidelity (with input F = 0.7) and corresponding symbolic expression for the Fidelity
def find_F_max(polies):
    Fmax = 0
    for i in polies:
        p, fid = symb_poly(i)
    #     G = var('G')
    #     k = fid.substitute(F=(1-G))
    #     series = k.taylor(G, 0, 4)
    #     coef = series.coefficients(sparse=False)
        Fnum = fid(F = 0.7)

        if Fnum > Fmax:
            Fmax = Fnum
            fidmax = fid
    print('---')
    print('Max fidelity = ', Fmax)
    print('Max fidelity (symbolic) =', fidmax)
    
def find_specific_F(polies, Ftarget):
    for i in polies:
        p, fid = symb_poly(i)

        Fnum = round(fid(0.7), 4)
        if Fnum == Ftarget:
            return i
    print(af)

In [606]:
polies2 = load('p+F+a+b+B+n2.sobj')
polies3 = load('p+F+a+b+B+n3.sobj')
polies4 = load('p+F+a+b+B+n4.sobj')
polies5 = load('p+F+a+b+B+n5.sobj')
polies6 = load('p+F+a+b+B+n6.sobj')
polies7 = load('p+F+a+b+B+n7.sobj')
polies8 = load('p+F+a+b+B+n8.sobj')

polydejmps2 = find_specific_F(polies2, 0.7353)
polydejmps3 = find_specific_F(polies3, 0.7857)
polydejmps4 = find_specific_F(polies4, 0.8459)
polydejmps5 = find_specific_F(polies5, 0.8636)
polydejmps6 = find_specific_F(polies6, 0.8982)
polydejmps7 = find_specific_F(polies7, 0.9091)
polydejmps8 = find_specific_F(polies8, 0.9344)

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:9: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:10: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:11: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and w

In [7]:
# all optimal dejmps protocols
def load_dejmps_data():
    polydejmps2 = ((1, 0, 1), (5, 2, 1))
    polydejmps3 = ((2, 1, 0, 1), (7, 7, 1, 1))
    polydejmps4 = ((5, 0, 2, 0, 1), (13, 8, 10, 0, 1))
    polydejmps5 = ((6, 5, 2, 2, 0, 1), (18, 23, 14, 8, 0, 1))
    polydejmps6 = ((5, 16, 7, 0, 3, 0, 1), (31, 40, 27, 24, 5, 0, 1))
    polydejmps7 = ((0, 47, 0, 11, 0, 5, 0, 1), (60, 47, 96, 11, 36, 5, 0, 1))
    polydejmps8 = ((25, 0, 84, 0, 14, 0, 4, 0, 1), (89, 80, 212, 32, 78, 16, 4, 0, 1))
    return [polydejmps2, polydejmps3, polydejmps4, polydejmps5, polydejmps6, polydejmps7, polydejmps8]

In [230]:
F = var('F')

poly1 = (F)
poly2 = (10*F^2 - 2*F + 1)/(8*F^2 - 4*F + 5)
poly3 = (14*F^2 - 7*F + 2)/(16*F^2 - 14*F + 7)
poly4 = (6*F^2 - 4*F + 1)/(8*F^2 - 8*F + 3)
poly5 = (32*F^5 - 20*F^4 + 30*F^3 - 20*F^2 + 5*F)/(80*F^4 - 80*F^3 + 30*F^2 - 5*F + 2)
poly6 = (144*F^5 - 128*F^4 + 104*F^3 - 48*F^2 + 8*F + 1)/(64*F^5 + 128*F^4 - 192*F^3 + 104*F^2 - 32*F + 9)
poly7 = (1184*F^6 - 888*F^5 + 738*F^4 - 391*F^3 + 96*F^2 - 18*F + 8)/(1024*F^6 - 576*F^5 + 768*F^4 - 782*F^3 + 369*F^2 - 111*F + 37)
poly8 = (1696*F^6 - 1824*F^5 + 1392*F^4 - 728*F^3 + 240*F^2 - 60*F + 13)/(1664*F^6 - 1920*F^5 + 1920*F^4 - 1456*F^3 + 696*F^2 - 228*F + 53)

F_min = 0.8

plot1 = plot(poly1, (F_min, 1), thickness=1.0, rgbcolor=(0, 1, 1))
plot2 = plot(poly2, (F_min, 1), thickness=1.0, rgbcolor=(0.1, 0.9, 1))
plot3 = plot(poly3, (F_min, 1), thickness=1.0, rgbcolor=(0.2, 0.8, 1))
plot4 = plot(poly4, (F_min, 1), thickness=1.0, rgbcolor=(0.3, 0.7, 1))
plot5 = plot(poly5, (F_min, 1), thickness=1.0, rgbcolor=(0.4, 0.6, 1))
plot6 = plot(poly6, (F_min, 1), thickness=1.0, rgbcolor=(0.5, 0.5, 1))
plot7 = plot(poly7, (F_min, 1), thickness=1.0, rgbcolor=(0.6, 0.4, 1))
plot8 = plot(poly8, (F_min, 1), thickness=1.0, rgbcolor=(0.7, 0.3, 1))

plot_all = plot1+plot2+plot3+plot4+plot5+plot6+plot7+plot8
plot_all.set_axes_range(xmin=F_min, xmax=1)
save(plot_all,'fidelities_from_n_1_to_8_zoom.png',axes=True, axes_labels=['$F_{in}$','$F_{out}$']) 



In [401]:
DEJMPS4 = (104*F^4 - 56*F^3 + 48*F^2 - 20*F + 5)/(160*F^4 - 160*F^3 + 96*F^2 - 28*F + 13)
doubleDEJMPS2 = (25 - 52*F + 204*F^2 - 352*F^3 + 904*F^4)/(113 - 176*F + 408*F^2 - 416*F^3 + 800*F^4)
poly4 = (6*F^2 - 4*F + 1)/(8*F^2 - 8*F + 3)
Fmin = 1/2
plot_DEJMPS4 = plot(DEJMPS4, (Fmin, 1), thickness=1.0, rgbcolor=(0, 1, 1), legend_label='DEJMPS with rotations')
plot_doubleDEJMPS2 = plot(doubleDEJMPS2, (Fmin, 1), thickness=1.0, rgbcolor=(0.6, 0.3, 1), legend_label = 'DEJMPS with twirling')
plot4 = plot(poly4, (Fmin, 1), thickness=1.0, rgbcolor=(0.3, 0.7, 1), legend_label = 'Optimal protocol')

plot_DEJMPS_comp4 = plot4+plot_DEJMPS4+plot_doubleDEJMPS2
plot_DEJMPS_comp4.set_axes_range(xmin=Fmin, xmax=1)
save(plot_DEJMPS_comp4,'plot_DEJMPS_comp4.pdf',axes=True, axes_labels=['$F_{in}$','$F_{out}$']) 

In [11]:
# calc entropy of coefficients of Werner state with fidelity F
def calc_entropy(F):
    return -F*log(F)/log(2)-(1-F)*log((1-F)/3)/log(2)

def calc_rate(poly, n):
    p, F = symb_poly(poly)
    
    return max_symbolic(p*(1-calc_entropy(F))/n, 0)

def bin_entropy(p):
    if p == 0:
        return 0
    return -p*log(p)/log(2)-(1-p)*log(1-p)/log(2)

def calc_SKR(poly, n):
    p, F = symb_poly(poly)
    return max_symbolic(p*(1-2*bin_entropy(((1-F)/3)))/n, 0)

def load_data():
    polies2 = load('p+F+a+b+B+n2.sobj')
    polies3 = load('p+F+a+b+B+n3.sobj')
    polies4 = load('p+F+a+b+B+n4.sobj')
    polies5 = load('p+F+a+b+B+n5.sobj')
    polies6 = load('p+F+a+b+B+n6.sobj')
    polies7 = load('p+F+a+b+B+n7.sobj')
    polies8 = load('p+F+a+b+B+n8.sobj')

    list_of_polies = [polies2, polies3, polies4, polies5, polies6, polies7, polies8]
    list_of_n = [2, 3, 4, 5, 6, 7, 8]
    assert len(list_of_polies) == len(list_of_n)
    
    rgbcolors=[(0+x/(len(list_of_n)-1), 1-x/(len(list_of_n)-1), 1) for x in range(len(list_of_n))]
    return list_of_polies, list_of_n, rgbcolors
    
def plot_all_rates():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    rates = plot([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_rates = [calc_rate(poly, n) for poly in polies]
        max_rate = max_symbolic(list_of_rates)
        rates += plot(max_rate, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    rates.set_axes_range(xmin=1/2, xmax=1, ymin=0, ymax=1/2)
    save(rates,'max_all_rates.pdf',axes=True, axes_labels=['$F_{in}$','Rate']) 
    
def plot_all_rates_zoom():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    rates = plot_semilogy([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_rates = [calc_rate(poly, n) for poly in polies]
        max_rate = max_symbolic(list_of_rates)
        rates += plot_semilogy(max_rate, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    rates.set_axes_range(xmin=1/2, xmax=1, ymin=1e-4, ymax=1/2)
    save(rates,'max_all_rates_zoom.pdf',axes=True, axes_labels=['$F_{in}$','Rate']) 

    
def plot_all_rates_zoom_comp():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    rates = plot_semilogy([])
#     for i in range(len(list_of_n)):
#         polies = list_of_polies[i]
#         n = list_of_n[i]
#         list_of_rates = [calc_rate(poly, n) for poly in polies]
#         max_rate = max_symbolic(list_of_rates)
#         rates += plot_semilogy(max_rate, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    
    data = load_dejmps_data()
    for i, d in enumerate(data):
        max_fid = calc_rate(d, i+2)
        rates += plot_semilogy(max_fid, (Fmin, 1), linestyle=':', thickness=1.6, rgbcolor=rgbcolors[i])
    
    rates.set_axes_range(xmin=1/2, xmax=1, ymin=1e-4, ymax=1/2)
    save(rates,'max_all_rates_zoom_comp.pdf',axes=True, axes_labels=['$F_{in}$','Rate']) 

def plot_all_fidelities():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    fids = plot([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_fids = [symb_poly(poly)[1] for poly in polies]
        max_fidelity = max_symbolic(list_of_fids)
        print(n)
        print(max_fidelity(0.7))
        fids += plot(max_fidelity, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    max_fidelity = (14848*F^8 - 15104*F^7 + 11752*F^6 - 8312*F^5 + 5314*F^4 - 2660*F^3 + 889*F^2 - 200*F + 34)/(17408*F^8 - 25600*F^7 + 28448*F^6 - 21280*F^5 + 10628*F^4 - 4156*F^3 + 1469*F^2 - 472*F + 116)
    fids += plot(max_fidelity, (1/2, 1), rgbcolor=(2/3, 2/3, 1/3), legend_label='n = 10')
    data = load_dejmps_data()
    for i, d in enumerate(data):
        max_fid = symb_poly(d)[1]
        fids += plot(max_fid, (1/2, 1), linestyle=':', rgbcolor=rgbcolors[i])
    fids.set_axes_range(xmin=1/2, xmax=1, ymin=1/2, ymax=1)
    save(fids,'max_all_fidelities+10.pdf',axes=True, axes_labels=['$F_{in}$','$F_{out}$'])
    
def plot_all_fidelities_zoom():
    list_of_polies, list_of_n, rgbcolors = load_data()
    Fmin = 0.9
    fids = plot_semilogy([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_fids = [1-symb_poly(poly)[1] for poly in polies]
        max_fidelity = min_symbolic(list_of_fids)
        fids += plot_semilogy(max_fidelity, (Fmin, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    
    max_fidelity = 1-(14848*F^8 - 15104*F^7 + 11752*F^6 - 8312*F^5 + 5314*F^4 - 2660*F^3 + 889*F^2 - 200*F + 34)/(17408*F^8 - 25600*F^7 + 28448*F^6 - 21280*F^5 + 10628*F^4 - 4156*F^3 + 1469*F^2 - 472*F + 116)
    fids += plot_semilogy(max_fidelity, (Fmin, 1), rgbcolor=(2/3, 2/3, 1/3), legend_label='n = 10')
    
    data = load_dejmps_data()
    for i, d in enumerate(data):
        max_fid = 1-symb_poly(d)[1]
        fids += plot_semilogy(max_fid, (Fmin, 1), linestyle=':', thickness=1.6, rgbcolor=rgbcolors[i])
#     fids.set_axes_range(xmin=Fmin, xmax=1)
    fids.set_axes_range(xmin=Fmin, xmax=1, ymin=1e-7, ymax=0.1)
    save(fids,'max_all_fidelities_zoom+10.pdf',axes=True, axes_labels=['$F_{in}$','$1-F_{out}$'])
    

def plot_all_probabilities():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    probs = plot([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_probs = [symb_poly(poly)[0] for poly in polies]
        max_prob = min_symbolic(list_of_probs)
        probs += plot(max_prob, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    probs.set_axes_range(xmin=1/2, xmax=1, ymin=0, ymax=1)
    save(probs,'min_all_probabilities.pdf',axes=True, axes_labels=['$F_{in}$','Success probability'])
    
    
def plot_all_SKR():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    SKR = plot([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_SKR = [calc_SKR(poly, n) for poly in polies]
        max_SKR = max_symbolic(list_of_SKR)
        SKR += plot(max_SKR, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    SKR.set_axes_range(xmin=1/2, xmax=1)
    save(SKR,'max_all_SKR.pdf',axes=True, axes_labels=['$F_{in}$','SKR'])
    
def plot_all_fid_succ():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    fid_succs = plot([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_fid_succs = [symb_poly(poly) for poly in polies]
        list_of_fid_succs = [x[0]*x[1] for x in list_of_fid_succs]
        max_fid_succ = max_symbolic(list_of_fid_succs)
        fid_succs += plot(max_fid_succ, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    fid_succs.set_axes_range(xmin=1/2, xmax=1)
    save(fid_succs,'max_all_fid_succ.pdf',axes=True, axes_labels=['$F_{in}$','$F_{out} \cdot p$'])
    
    

In [12]:
plot_all_fidelities_zoom()
# plot_all_rates_zoom_comp()


In [429]:
def plot_dejmps_vs_opt():
    opt2 = (10*F^2 - 2*F + 1)/(8*F^2 - 4*F + 5)
    
    opt2 = (10*F^2 - 2*F + 1)/(8*F^2 - 4*F + 5)
    
    opt3 = (14*F^2 - 7*F + 2)/(16*F^2 - 14*F + 7)
    
    naive_dejmps4 = opt2(opt2)
    dejmps4 = (104*F^4 - 56*F^3 + 48*F^2 - 20*F + 5)/(160*F^4 - 160*F^3 + 96*F^2 - 28*F + 13)
    opt4 = (6*F^2 - 4*F + 1)/(8*F^2 - 8*F + 3)
    
    
    dejmps5 = (6*F^2 - 4*F + 1)/(8*F^2 - 8*F + 3)
    opt5 = (32*F^5 - 20*F^4 + 30*F^3 - 20*F^2 + 5*F)/(80*F^4 - 80*F^3 + 30*F^2 - 5*F + 2)
    
    naive_dejmps8 = opt2(opt2(opt2))
    dejmps8 = (11392*F^8 - 15104*F^7 + 21760*F^6 - 21056*F^5 + 14224*F^4 - 5936*F^3 + 1456*F^2 - 200*F + 25)/(12800*F^8 - 25600*F^7 + 46592*F^6 - 47488*F^5 + 28448*F^4 - 10528*F^3 + 2720*F^2 - 472*F + 89)
    opt8 = (1696*F^6 - 1824*F^5 + 1392*F^4 - 728*F^3 + 240*F^2 - 60*F + 13)/(1664*F^6 - 1920*F^5 + 1920*F^4 - 1456*F^3 + 696*F^2 - 228*F + 53)
    
    
    naive_dejmps4_plot = plot(naive_dejmps4, (1/2, 1), rgbcolor=(0, 1, 1), legend_label='n = 4, naive', linestyle=':')
    dejmps4_plot = plot(dejmps4, (1/2, 1), rgbcolor=(0, 1, 1), legend_label='n = 4, DEJMPS', linestyle='--')
    opt4_plot = plot(opt4, (1/2, 1), rgbcolor=(0, 1, 1), legend_label='n = 4, optimal')
    
    dejmps5_plot = plot(dejmps5, (1/2, 1), rgbcolor=(1/2, 1/2, 1), legend_label='n = 5, DEJMPS', linestyle='--')
    opt5_plot = plot(opt5, (1/2, 1), rgbcolor=(1/2, 1/2, 1), legend_label='n = 5, optimal')
    
    naive_dejmps8_plot = plot(naive_dejmps8, (1/2, 1), rgbcolor=(1, 0, 1), legend_label='n = 8, naive', linestyle=':')
    dejmps8_plot = plot(dejmps8, (1/2, 1), rgbcolor=(1, 0, 1), legend_label='n = 8, DEJMPS', linestyle='--')
    opt8_plot = plot(opt8, (1/2, 1), rgbcolor=(1, 0, 1), legend_label='n = 8, optimal')
    
    dejmps_vs_opt = naive_dejmps4_plot + dejmps4_plot + opt4_plot + dejmps5_plot + opt5_plot + naive_dejmps8_plot + dejmps8_plot + opt8_plot
    dejmps_vs_opt.set_axes_range(xmin=1/2, xmax=1)
    save(dejmps_vs_opt,'dejmps_vs_opt2.pdf',axes=True, axes_labels=['$F_{in}$','$F_{out}$'])

plot_dejmps_vs_opt()



/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:36: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
